In [ ]:
# !pip install keras_cv_attention_models

In [2]:
import evals
from tensorflow import keras
import tensorflow as tf
#import IJB_evals
import matplotlib.pyplot as plt
import keras_cv_attention_models
import GhostFaceNets, GhostFaceNets_with_Bias

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

2 ways to load the model

Stride 2

In [3]:
#Either
basic_model = keras.models.load_model('checkpoints/GN_W1.3_S1_ArcFace_epoch46.h5', compile=False)

OSError: No file or directory found at checkpoints/ghostnetv1_w1.3_s2.h5

In [ ]:
#If the above did not work, then you need to build the model then load the weights. e.g.,
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model, target_activation='PReLU')

basic_model.load_weights('checkpoints/ghostnetv1_w1.3_s2.h5')

Stride 1

In [ ]:
#Either
basic_model = keras.models.load_model('checkpoints/ghostnetv1_w1.3_s1.h5', compile=False)

In [ ]:
#If the above did not work, then you need to build the model then load the weights. e.g.,
basic_model = GhostFaceNets_with_Bias.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets_with_Bias.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets_with_Bias.replace_ReLU_with_PReLU(basic_model, target_activation='PReLU')

basic_model.load_weights('checkpoints/ghostnetv1_w1.3_s1.h5')

In [12]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=True)
ee.on_epoch_end(0)

Evaluating lfw: 100%|██████████| 47/47 [00:39<00:00,  1.18it/s]



>>>> lfw evaluation max accuracy: 0.996833, thresh: 0.223459, previous max accuracy: 0.000000, PCA accuray = 0.996000 ± 0.002494
>>>> Improved = 0.996833


In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/lfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/vgg2_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cfp_ff.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cfp_fp.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/calfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/cplfw.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
ee = evals.eval_callback(basic_model, 'datasets/faces_emore/agedb_30.bin', batch_size=256, flip=True, PCA_acc=False)
ee.on_epoch_end(0)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='path_to_IJB_Dataset/ijb-testsuite/ijb', subset='IJBB', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBB"], label=tt.label)

In [ ]:
tt = IJB_evals.IJB_test(lambda imgs: basic_model((tf.cast(imgs, "float32") - 127.5) * 0.0078125).numpy(), data_path='C:/Users/mohda/Downloads/ijb-testsuite/ijb', subset='IJBC', batch_size=16)

In [ ]:
score = tt.run_model_test_single()

In [ ]:
IJB_evals.plot_roc_and_calculate_tpr([score], names=[basic_model.name + "_IJBC"], label=tt.label)

Plot face quality distribution using norm value of feature

In [ ]:
cc = tf.norm(ee.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='agedb_30 quality')
cc = tf.norm(tt.embs, axis=1).numpy()
_ = plt.hist(cc, bins=512, alpha=0.5, label='IJBC quality')
plt.legend()
plt.tight_layout()